In [1]:
import magma as m
from functools import reduce
m.set_mantle_target("coreir")
import mantle

def one_hot_mux(conds, inputs):
    outputs = []
    for cond, inp in zip(conds, inputs):
        outputs.append(inp & m.bits([cond for _ in range(len(inp))]))
    return reduce(lambda x, y: x | y, outputs)


class SimpleALU(m.Circuit):
    name = "SimpleALU"
    IO = ["a", m.In(m.UInt(4)), "b", m.In(m.UInt(4)), 
          "opcode", m.In(m.UInt(2)), "out", m.Out(m.UInt(4))]
    
    @classmethod
    def definition(io):
        is_op0 = mantle.eq(io.opcode, m.uint(0, n=2))
        is_op1 = mantle.eq(io.opcode, m.uint(1, n=2))
        is_op2 = mantle.eq(io.opcode, m.uint(2, n=2))
        is_op3 = mantle.eq(io.opcode, m.uint(3, n=2))
        op0_out = io.a + io.b
        op1_out = io.a - io.b
        op2_out = io.a
        op3_out = io.b
        m.wire(io.out, one_hot_mux([is_op0, is_op1, is_op2, is_op3], [op0_out, op1_out, op2_out, op3_out]))

In [2]:
from magma.backend.verilog import compile as compile_verilog
print(compile_verilog(SimpleALU))

compiling EQ2
compiling Add4
compiling Invert4_wrapped
compiling Add4_cin
compiling Sub4
compiling and4_wrapped
compiling or4_wrapped
compiling SimpleALU
module EQ2 (input [1:0] I0, input [1:0] I1, output  O);
wire  inst0_out;
coreir_eq inst0 (.in0(I0), .in1(I1), .out(inst0_out));
assign O = inst0_out;
endmodule

module Add4 (input [3:0] I0, input [3:0] I1, output [3:0] O);
wire [3:0] inst0_out;
coreir_add4 inst0 (.in0(I0), .in1(I1), .out(inst0_out));
assign O = inst0_out;
endmodule

module Invert4_wrapped (input [3:0] I, output [3:0] O);
wire [3:0] inst0_out;
coreir_not inst0 (.in(I), .out(inst0_out));
assign O = inst0_out;
endmodule

module Add4_cin (input [3:0] I0, input [3:0] I1, output [3:0] O, input  CIN);
wire [3:0] inst0_out;
wire [3:0] inst1_out;
coreir_add4 inst0 (.in0(inst1_out), .in1(I1), .out(inst0_out));
coreir_add4 inst1 (.in0({1'b0,1'b0,1'b0,CIN}), .in1(I0), .out(inst1_out));
assign O = inst0_out;
endmodule

module Sub4 (input [3:0] I0, input [3:0] I1, output [3:0] O);


In [3]:
from magma.backend.verilog import compile as compile_verilog

print(compile_verilog(SimpleALU))

compiling EQ2
compiling Add4
compiling Invert4_wrapped
compiling Add4_cin
compiling Sub4
compiling and4_wrapped
compiling or4_wrapped
compiling SimpleALU
module EQ2 (input [1:0] I0, input [1:0] I1, output  O);
wire  inst0_out;
coreir_eq inst0 (.in0(I0), .in1(I1), .out(inst0_out));
assign O = inst0_out;
endmodule

module Add4 (input [3:0] I0, input [3:0] I1, output [3:0] O);
wire [3:0] inst0_out;
coreir_add4 inst0 (.in0(I0), .in1(I1), .out(inst0_out));
assign O = inst0_out;
endmodule

module Invert4_wrapped (input [3:0] I, output [3:0] O);
wire [3:0] inst0_out;
coreir_not inst0 (.in(I), .out(inst0_out));
assign O = inst0_out;
endmodule

module Add4_cin (input [3:0] I0, input [3:0] I1, output [3:0] O, input  CIN);
wire [3:0] inst0_out;
wire [3:0] inst1_out;
coreir_add4 inst0 (.in0(inst1_out), .in1(I1), .out(inst0_out));
coreir_add4 inst1 (.in0({1'b0,1'b0,1'b0,CIN}), .in1(I0), .out(inst1_out));
assign O = inst0_out;
endmodule

module Sub4 (input [3:0] I0, input [3:0] I1, output [3:0] O);


In [4]:
from magma.backend.coreir_ import compile as compile_coreir

compile_coreir(SimpleALU, "build/SimpleALU.json")
with open("build/SimpleALU.json", "r") as f:
    print(f.read())

{"top":"global.SimpleALU",
"namespaces":{
  "global":{
    "modules":{
      "Add4":{
        "type":["Record",[
          ["I0",["Array",4,"BitIn"]],
          ["I1",["Array",4,"BitIn"]],
          ["O",["Array",4,"Bit"]]
        ]],
        "instances":{
          "inst0":{
            "genref":"coreir.add",
            "genargs":{"width":["Int",4]}
          }
        },
        "connections":[
          ["self.I0","inst0.in0"],
          ["self.I1","inst0.in1"],
          ["self.O","inst0.out"]
        ]
      },
      "Add4_cin":{
        "type":["Record",[
          ["I0",["Array",4,"BitIn"]],
          ["I1",["Array",4,"BitIn"]],
          ["O",["Array",4,"Bit"]],
          ["CIN","BitIn"]
        ]],
        "instances":{
          "bit_const_GND":{
            "modref":"corebit.const",
            "modargs":{"value":["Bool",false]}
          },
          "inst0":{
            "genref":"coreir.add",
            "genargs":{"width":["Int",4]}
          },
          "inst1":{
    

In [5]:
from magma.simulator import PythonSimulator
from magma.bit_vector import BitVector

simulator = PythonSimulator(SimpleALU)
simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(0, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(3 + 2, num_bits=4), simulator.get_value(SimpleALU.out)

simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(1, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(3 - 2, num_bits=4)
simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(2, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(3, num_bits=4)

simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(3, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(2, num_bits=4)
print("Success!")

Success!
